# import

In [2]:
from nexus_ai.ABSA.arabic.bert import bert_ATE, bert_APC
from nexus_ai.ABSA.arabic.dataset import dataset_ATE, dataset_APC
from nexus_ai.sentence_sentiment_analysis.preprocessing import pad_features


In [3]:
import random
from torch.utils.data import DataLoader
from transformers import BertTokenizer
import torch
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from IPython.utils import io

In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# for debugin
# DEVICE = "cpu"
pretrain_model_name = "../../sentence_sentiment_analysis/arabic/models/bert_pretrained_01_acc_90.50"
# pretrain_model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa"
tokenizer = BertTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")

In [5]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model
    
def save_model(model, name):
    torch.save(model.state_dict(), name)

In [6]:
def set_seed(seed_value=12):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

# Acpect Term Extraction

In [7]:
train_ds = dataset_ATE(pd.read_csv("dataset/arabic_train.csv"), tokenizer)
test_ds = dataset_ATE(pd.read_csv("dataset/arabic_test.csv"), tokenizer)

In [8]:
w,x,y,z = train_ds.__getitem__(3)
print(w)
print(len(w))
print(x)
print(x.size())
print(y)
print(y.size())
print(z)
print(z.size())

['ساو', '##صي', 'بالت', '##اكيد', 'بموقع', 'المدينه', 'القديمه', 'الا', 'انه', 'عليك', 'الحذر', 'من', 'الاسعار', 'السياحي', '##ه', 'الاكثر', 'ارتفاعا', '.']
18
tensor([23065,  3389,  2619, 23131, 19176,  8322, 15121,  1915,  2615,  2898,
        17133,  1908,  9384, 13931,  1028,  9442, 23596,    18])
torch.Size([18])
tensor([0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0])
torch.Size([18])
tensor([-1, -1, -1, -1, -1,  2,  2, -1, -1, -1, -1, -1,  2,  2,  2, -1, -1, -1])
torch.Size([18])


In [10]:
print(train_ds)

In [16]:
def create_mini_batch(samples):
    print(samples)
    ids_tensors = [s[1] for s in samples]
    ids_tensors = torch.Tensor(pad_features(ids_tensors, 128)).long()

    tags_tensors = [s[2] for s in samples]
    tags_tensors = torch.Tensor(pad_features(tags_tensors, 128)).long()

    pols_tensors = [s[3] for s in samples]
    pols_tensors = torch.Tensor(pad_features(pols_tensors, 128)).long()
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
    
    return ids_tensors, tags_tensors, pols_tensors, masks_tensors

In [17]:
train_loader = DataLoader(train_ds, batch_size=16, collate_fn=create_mini_batch, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle = True)

In [18]:
for batch in train_loader:
    w,x,y,z = batch
    print(w)
    print(w.size())
    print(x)
    print(x.size())
    print(y)
    print(y.size())
    print(z)
    print(z.size())
    break

[(['موقع', 'جيد', '،', 'فندق', 'لطيف', '،', 'عامل', '##ون', 'ود', '##ودو', '##ن', 'لقد', 'قضي', '##نا', 'وقتا', 'رائعا', 'في', 'فوك', '##يت', 'لر', '##حله', 'ذكري', 'زواج', '##نا'], tensor([ 2867,  6842,   378,  7073, 14745,   378,  7780,  1922,  2557, 19189,
         1006,  3863, 25218,  1914, 18313, 23526,  1912, 18068,  2034,  5165,
        16801,  5721,  9000,  1914]), tensor([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([ 2, -1, -1,  2, -1, -1,  2,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1])), (['مه', '##جور', 'الاسره', 'غير', 'مريح', '##ه', 'للغ', '##ايه', 'وهي', 'ليست', 'باس', '##ره', 'علي', 'الاطلاق', 'في', 'واقع', 'الامر', '–', 'اشبه', 'بالار', '##يك', '##ه', 'الكبيره', 'المت', '##ها', '##لكه'], tensor([ 3349, 16573, 19380,  2224, 24566,  1028,  4740, 11890,  2843,  3721,
         6736,  2197,  1958, 19263,  1912,  6422,  5122,   616, 22606, 27857,
         2434,  1028, 19318,  2087,  1913,  4194]), tensor([0

In [10]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)
            
            loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ATE.step()
            optimizer_ATE.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_ATE, 'models/bert_ATE.pt')
        
def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            _, predictions = torch.max(outputs, dim=2)

            pred += list([int(j) for i in predictions for j in i ])
            trueth += list([int(j) for i in tags_tensors for j in i ])

    return trueth, pred



In [70]:
set_seed(12)
lr_list = [
    1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 7e-4, 8e-4, 9e-4,
    1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5,
    1e-6, 2e-6, 3e-6, 4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6,
]
for lr in lr_list:
    torch.cuda.empty_cache()
    with io.capture_output() as captured:
        model_ATE = bert_ATE(pretrain_model_name, local_files_only=True).to(DEVICE)
        optimizer_ATE = torch.optim.AdamW(model_ATE.parameters(), lr=lr)
        train_model_ATE(train_loader, 2)
    %time x, y = test_model_ATE(test_loader)
    print(f'learning rate: {lr}')
    print(classification_report(x, y, target_names=[str(i) for i in range(3)]))
    torch.cuda.empty_cache()


CPU times: total: 11.8 s
Wall time: 11.8 s
learning rate: 0.0001
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    116071
           1       0.77      0.66      0.71      1791
           2       0.56      0.57      0.56       410

    accuracy                           0.99    118272
   macro avg       0.77      0.74      0.76    118272
weighted avg       0.99      0.99      0.99    118272

CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 0.0002
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    116071
           1       0.69      0.73      0.71      1791
           2       0.48      0.67      0.56       410

    accuracy                           0.99    118272
   macro avg       0.72      0.80      0.75    118272
weighted avg       0.99      0.99      0.99    118272

CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 0.0003
              precision    recall  f1-score

c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 0.0004
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    116071
           1       0.00      0.00      0.00      1791
           2       0.00      0.00      0.00       410

    accuracy                           0.98    118272
   macro avg       0.33      0.33      0.33    118272
weighted avg       0.96      0.98      0.97    118272



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 0.0005
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    116071
           1       0.00      0.00      0.00      1791
           2       0.00      0.00      0.00       410

    accuracy                           0.98    118272
   macro avg       0.33      0.33      0.33    118272
weighted avg       0.96      0.98      0.97    118272



...

CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 3e-06
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    116071
           1       0.00      0.00      0.00      1791
           2       0.00      0.00      0.00       410

    accuracy                           0.98    118272
   macro avg       0.33      0.33      0.33    118272
weighted avg       0.96      0.98      0.97    118272



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 4e-06
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    116071
           1       0.54      0.01      0.02      1791
           2       0.00      0.00      0.00       410

    accuracy                           0.98    118272
   macro avg       0.51      0.34      0.34    118272
weighted avg       0.97      0.98      0.97    118272



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 5e-06
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    116071
           1       0.59      0.12      0.19      1791
           2       1.00      0.00      0.01       410

    accuracy                           0.98    118272
   macro avg       0.86      0.37      0.40    118272
weighted avg       0.98      0.98      0.98    118272

CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 6e-06
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    116071
           1       0.60      0.32      0.42      1791
           2       0.71      0.03      0.06       410

    accuracy                           0.98    118272
   macro avg       0.77      0.45      0.49    118272
weighted avg       0.98      0.98      0.98    118272

CPU times: total: 11.7 s
Wall time: 11.7 s
learning rate: 7e-06
              precision    recall  f1-score   

In [15]:
lr = 5e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.AdamW(model_ATE.parameters(), lr=lr)

In [16]:
train_model_ATE(train_loader, 2)

epoch: 0  batch: 1 / 226  loss: 1.0914989709854126  hr: 0  min: 0  sec: 46
epoch: 0  batch: 2 / 226  loss: 0.702249675989151  hr: 0  min: 0  sec: 45
epoch: 0  batch: 3 / 226  loss: 0.5087520902355512  hr: 0  min: 0  sec: 48
epoch: 0  batch: 4 / 226  loss: 0.40624653920531273  hr: 0  min: 0  sec: 49
epoch: 0  batch: 5 / 226  loss: 0.3497561901807785  hr: 0  min: 0  sec: 50
epoch: 0  batch: 6 / 226  loss: 0.30826083322366077  hr: 0  min: 0  sec: 50
epoch: 0  batch: 7 / 226  loss: 0.28083924842732294  hr: 0  min: 0  sec: 51
epoch: 0  batch: 8 / 226  loss: 0.2630830826237798  hr: 0  min: 0  sec: 51
epoch: 0  batch: 9 / 226  loss: 0.2494450898634063  hr: 0  min: 0  sec: 51
epoch: 0  batch: 10 / 226  loss: 0.2309298887848854  hr: 0  min: 0  sec: 51
epoch: 0  batch: 11 / 226  loss: 0.22354887832294812  hr: 0  min: 0  sec: 51
epoch: 0  batch: 12 / 226  loss: 0.21553566182653108  hr: 0  min: 0  sec: 52
epoch: 0  batch: 13 / 226  loss: 0.2079309901365867  hr: 0  min: 0  sec: 52
epoch: 0  batch: 

In [21]:
model_ATE = load_model(model_ATE, 'models/bert_ATE.pt')

In [22]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: total: 12 s
Wall time: 12 s
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    116071
           1       0.72      0.76      0.74      1791
           2       0.60      0.57      0.58       410

    accuracy                           0.99    118272
   macro avg       0.77      0.77      0.77    118272
weighted avg       0.99      0.99      0.99    118272



# Aspect polarity classification (APC)

In [6]:
train_ds = dataset_APC(pd.read_csv("dataset/arabic_train.csv"), tokenizer)
test_ds = dataset_APC(pd.read_csv("dataset/arabic_test.csv"), tokenizer)

In [7]:
w,x,y,z = train_ds.__getitem__(3)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'ساو', '##صي', 'بالت', '##اكيد', 'بموقع', 'المدينه', 'القديمه', 'الا', 'انه', 'عليك', 'الحذر', 'من', 'الاسعار', 'السياحي', '##ه', 'الاكثر', 'ارتفاعا', '.', '[sep]', 'المدينه', 'القديمه', 'الاسعار', 'السياحي', '##ه']
25
tensor([    1, 23065,  3389,  2619, 23131, 19176,  8322, 15121,  1915,  2615,
         2898, 17133,  1908,  9384, 13931,  1028,  9442, 23596,    18,     1,
         8322, 15121,  9384, 13931,  1028])
25
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1])
25
tensor(2)


In [8]:
def create_mini_batch2(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = torch.Tensor(pad_features(ids_tensors, 128)).long()

    segments_tensors = [s[2] for s in samples]
    segments_tensors = torch.Tensor(pad_features(segments_tensors, 128)).long()

    label_ids = torch.stack([s[3] for s in samples])

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [9]:
train_loader = DataLoader(train_ds, batch_size=16, collate_fn=create_mini_batch2, shuffle = True)
test_loader = DataLoader(test_ds, batch_size=50, collate_fn=create_mini_batch2, shuffle = True)

In [17]:
# for batch in train_loader:
#     w,x,y,z = batch
#     print(w)
#     print(w.size())
#     print(x)
#     print(x.size())
#     print(y)
#     print(y.size())
#     print(z)
#     print(z.size())
#     break

In [12]:
def train_model_APC(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_APC(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_APC.step()
            optimizer_APC.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_APC, 'models/bert_APC.pt')
        
def test_model_APC(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_APC(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred



In [30]:
set_seed(12)
lr_list = [
    1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 7e-4, 8e-4, 9e-4,
    1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5,
    1e-6, 2e-6, 3e-6, 4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6,
]
for lr in lr_list:
    torch.cuda.empty_cache()
    with io.capture_output() as captured:
        model_APC = bert_APC(pretrain_model_name).to(DEVICE)
        optimizer_APC = torch.optim.AdamW(model_APC.parameters(), lr=lr)
        train_model_APC(train_loader, 2)
    %time x, y = test_model_APC(test_loader)
    print(f'learning rate: {lr}')
    print(classification_report(x, y, target_names=[str(i) for i in range(3)]))
    torch.cuda.empty_cache()


CPU times: total: 2.59 s
Wall time: 2.59 s
learning rate: 0.0001
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       464
           1       0.44      0.22      0.29        50
           2       0.85      0.92      0.88       410

    accuracy                           0.88       924
   macro avg       0.74      0.68      0.70       924
weighted avg       0.87      0.88      0.87       924

CPU times: total: 2.58 s
Wall time: 2.58 s
learning rate: 0.0002
              precision    recall  f1-score   support

           0       0.89      0.83      0.86       464
           1       0.00      0.00      0.00        50
           2       0.77      0.92      0.84       410

    accuracy                           0.83       924
   macro avg       0.55      0.58      0.57       924
weighted avg       0.79      0.83      0.80       924



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 2.53 s
Wall time: 2.53 s
learning rate: 0.0003
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       464
           1       0.00      0.00      0.00        50
           2       0.44      1.00      0.61       410

    accuracy                           0.44       924
   macro avg       0.15      0.33      0.20       924
weighted avg       0.20      0.44      0.27       924



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 2.52 s
Wall time: 2.5 s
learning rate: 0.0004
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       464
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00       410

    accuracy                           0.50       924
   macro avg       0.17      0.33      0.22       924
weighted avg       0.25      0.50      0.34       924



...

CPU times: total: 2.42 s
Wall time: 2.42 s
learning rate: 5e-06
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       464
           1       0.00      0.00      0.00        50
           2       0.81      0.93      0.87       410

    accuracy                           0.86       924
   macro avg       0.57      0.60      0.59       924
weighted avg       0.81      0.86      0.83       924



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 2.42 s
Wall time: 2.43 s
learning rate: 6e-06
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       464
           1       0.00      0.00      0.00        50
           2       0.84      0.89      0.86       410

    accuracy                           0.85       924
   macro avg       0.56      0.60      0.58       924
weighted avg       0.80      0.85      0.82       924



c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\meshari\nexus_ai\venv\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CPU times: total: 2.44 s
Wall time: 2.43 s
learning rate: 7e-06
              precision    recall  f1-score   support

           0       0.89      0.91      0.90       464
           1       0.50      0.06      0.11        50
           2       0.85      0.92      0.88       410

    accuracy                           0.87       924
   macro avg       0.75      0.63      0.63       924
weighted avg       0.85      0.87      0.85       924

CPU times: total: 2.42 s
Wall time: 2.43 s
learning rate: 8e-06
              precision    recall  f1-score   support

           0       0.88      0.91      0.90       464
           1       0.24      0.08      0.12        50
           2       0.86      0.90      0.88       410

    accuracy                           0.86       924
   macro avg       0.66      0.63      0.63       924
weighted avg       0.84      0.86      0.85       924

CPU times: total: 2.5 s
Wall time: 2.5 s
learning rate: 9e-06
              precision    recall  f1-score   su

In [10]:
lr = 2e-5
model_APC = bert_APC(pretrain_model_name).to(DEVICE) 
optimizer_APC = torch.optim.AdamW(model_APC.parameters(), lr=lr)

In [13]:
train_model_APC(train_loader, 2)

epoch: 0  batch: 1 / 226  loss: 1.1813442707061768  hr: 0  min: 16  sec: 10
epoch: 0  batch: 2 / 226  loss: 1.093517005443573  hr: 0  min: 9  sec: 3
epoch: 0  batch: 3 / 226  loss: 1.0027076800664265  hr: 0  min: 6  sec: 44
epoch: 0  batch: 4 / 226  loss: 0.9596229046583176  hr: 0  min: 5  sec: 34
epoch: 0  batch: 5 / 226  loss: 0.9134316563606262  hr: 0  min: 4  sec: 52
epoch: 0  batch: 6 / 226  loss: 0.8612467845280966  hr: 0  min: 4  sec: 24
epoch: 0  batch: 7 / 226  loss: 0.8169203911508832  hr: 0  min: 4  sec: 4
epoch: 0  batch: 8 / 226  loss: 0.8205685019493103  hr: 0  min: 3  sec: 48
epoch: 0  batch: 9 / 226  loss: 0.8196925123532613  hr: 0  min: 3  sec: 36
epoch: 0  batch: 10 / 226  loss: 0.7957261741161347  hr: 0  min: 3  sec: 26
epoch: 0  batch: 11 / 226  loss: 0.7882357022979043  hr: 0  min: 3  sec: 18
epoch: 0  batch: 12 / 226  loss: 0.7562475477655729  hr: 0  min: 3  sec: 11
epoch: 0  batch: 13 / 226  loss: 0.7399227504546826  hr: 0  min: 3  sec: 6
epoch: 0  batch: 14 / 22

In [21]:
model_ABSA = load_model(model_APC, 'models/bert_APC.pt')

In [22]:
%time x, y = test_model_APC(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

CPU times: total: 2.59 s
Wall time: 2.59 s
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       464
           1       0.42      0.26      0.32        50
           2       0.90      0.87      0.89       410

    accuracy                           0.88       924
   macro avg       0.74      0.70      0.71       924
weighted avg       0.87      0.88      0.87       924

